# Setting Up

FastText docs [here](https://fasttext.cc/docs/en/unsupervised-tutorial.html)

## Link notebook to drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Move to base folder

In [2]:
#  !!!! ES NECESARIO EDITAR CON EL PATH CORRESPONDIENTE  !!!!
%cd /content/gdrive/MyDrive/Colab Notebooks/TTT-2020-1-16

/content/gdrive/MyDrive/Colab Notebooks/TTT-2020-1-16


In [5]:
# Make sure target files exist
!mkdir 'dataset_files/Encoded/SINGLE-VEC/7FOLDS'
!mkdir 'dataset_files/Encoded/EMB-SEQ/7FOLDS'

mkdir: cannot create directory ‘dataset_files/Encoded/SINGLE-VEC/7FOLDS’: File exists
mkdir: cannot create directory ‘dataset_files/Encoded/EMB-SEQ/7FOLDS’: File exists


In [4]:
%pwd

'/content/gdrive/MyDrive/Colab Notebooks/TTT-2020-1-16'

## Install fast-text module

In [ ]:
!pip install fasttext
import fasttext

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093591 sha256=5c397ff5b90af4e0d2fb068b88e8621bdd099b5ff8d6b1d5780ac948e4229b9d
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


# Importar dataset preprocesado

### Train set

In [ ]:
# 7 FOLDS SPLITTING
import pandas as pd
train_df = pd.read_pickle('./dataset_files/preprocessed_train_dataset_7_folds.data', None)

In [ ]:
train_df.head()

,id,text,preprocessed_text_1,preprocessed_text_2,HS,TR,AG,HTA,kfold
0,22083,"@pnique Inmigrante motorizado,eres una rata de...","[nombre, inmigrante, motorizadoeres, una, rata...","[nombre, inmigrante, motorizadoeres, una, rata...",1,1,1,4,0
1,20095,la sacaron justo a l dia siguiente del documen...,"[la, sacaron, justo, a, l, dia, siguiente, del...","[la, sacaron, justo, a, l, dia, siguiente, del...",0,0,0,0,0
2,22568,@FioreSalo Callate vs puta,"[nombre, callate, vs, puta]","[nombre, callate, vs, puta]",1,1,1,4,0
3,22107,@Pontifex_es @Pontifex @valealazraki precisame...,"[nombre, nombre, nombre, precisamente, en, mi,...","[nombre, nombre, nombre, precisamente, en, mi,...",0,0,0,0,0
4,20895,QUE haces con cc mogolica pili — Puta tomi cal...,"[que, haces, con, cc, mogolica, pili, puta, to...","[que, haces, con, cc, mogolica, pili, puta, to...",1,1,1,4,0


### Test set

In [ ]:
import pandas as pd
test_df = pd.read_pickle('./dataset_files/preprocessed_test_dataset.data', None)

In [ ]:
test_df.shape

(1600, 8)

In [ ]:
test_df.head()

,id,text,preprocessed_text_1,preprocessed_text_2,HS,TR,AG,HTA
0,30344,#CadaMañana cállate la puta que me pario Kohan...,"[htg, cállate, la, puta, que, me, pario, kohan...","[cállate, la, puta, que, me, pario, kohan, vos...",0,0,0,0
1,30466,Estas navidades mi polla mereces,"[estas, navidades, mi, polla, mereces]","[estas, navidades, mi, polla, mereces]",0,0,0,0
2,31084,Si no aprobas te pego una cojida que no te la ...,"[si, no, aprobas, te, pego, una, cojida, que, ...","[si, no, aprobas, te, pego, una, cojida, que, ...",1,1,1,4
3,34552,"@AlecitoGamer @falklands_utd A, no entendiste ...","[nombre, nombre, a, no, entendiste, nada, ud, ...","[nombre, nombre, a, no, entendiste, nada, ud, ...",0,0,0,0
4,32538,"@deborahhq1973 Es normal, en Cataluña los Inde...","[nombre, es, normal, en, cataluña, los, indepe...","[nombre, es, normal, en, cataluña, los, indepe...",1,0,1,2


## longitud máxima de los docs preprocesados

In [ ]:
max_len = 0
larger_than_x = 0
x = 57
for entry in train_df.preprocessed_text_1.values.tolist():
  if len(entry)> x:
    larger_than_x += 1
  max_len = max(max_len, len(entry))

max_len, larger_than_x

(60, 5)

# Utils

## Model retrieving

In [ ]:
def ft_model(model_id):
  print('loading fatText model...')

  if model_id == 1:
    #FastText 1
    ft = fasttext.load_model('./Embeddings/Pretrained_Embeddings/FastText_1/cc.es.300.bin')
  elif model_id == 2:
    #FastText 2
    ft = fasttext.load_model('./Embeddings/Pretrained_Embeddings/FastText_2/wiki.es.bin')
  elif model_id == 3:
    #FastText 3
    ft = fasttext.load_model('./Embeddings/Pretrained_Embeddings/FastText_3/embeddings-l-model.bin') 

  return ft


## Encoding Functions

In [ ]:
import numpy as np
MAX_WORDS = 55
EMBEDDINGS_SIZE = 300

def transformTweets(preprocessed_tweets, format, ft_model):

  n_tweets = len(preprocessed_tweets)

  if format == 'SINGLE-VEC':
    print('Transforming tweets...')
    n_tweets = len(preprocessed_tweets)
    encoded_tweets = np.zeros((n_tweets, EMBEDDINGS_SIZE))

    for i, tweet in enumerate(preprocessed_tweets):
      encoded_tweets[i] = toSingleVectorEmbedding(tweet,ft_model)

    print("Tweets were succesfully transformed.")
    return encoded_tweets

  elif format == 'EMB-SEQ':
    print('Transforming tweets...')
    n_tweets = len(preprocessed_tweets)
    encoded_tweets = np.zeros((n_tweets, MAX_WORDS, EMBEDDINGS_SIZE))

    for i, tweet in enumerate(preprocessed_tweets):
      encoded_tweets[i] = toEmbedingsSequence(tweet,ft_model)

    print("Tweets were succesfully transformed.")
    return encoded_tweets

  else:
    print("Invalid format. (Expected 'SINGLE-VEC' or 'EMB-SEQ')")


def toSingleVectorEmbedding(preprocessed_tweet, ft_model):
  encoded_tweet = np.zeros((EMBEDDINGS_SIZE,))
  for w in preprocessed_tweet:
    word_vector = ft_model.get_word_vector(w)
    encoded_tweet = encoded_tweet + word_vector
  return encoded_tweet.reshape(1,-1)

def toEmbedingsSequence(preprocessed_tweet, ft_model):
  embeddings_sequence = np.zeros((1, MAX_WORDS, EMBEDDINGS_SIZE))
  for i, w in enumerate(preprocessed_tweet[:MAX_WORDS]):
    word_vector = ft_model.get_word_vector(w).reshape(1,-1)
    embeddings_sequence[0][i] = word_vector
  return embeddings_sequence

## encodeDataFolds()

In [ ]:
def encodeDataFolds(train_df, n_folds, prep_format, encoding_format, 
                    embeddings_model, embeddings_label):
  encoded_folds = []
  for K in range (n_folds):
    fold_mask = train_df.kfold==K
    fold_tweets = train_df.loc[fold_mask,f'preprocessed_text_{prep_format}']
    encoded_tweets = transformTweets(fold_tweets, 
                                    encoding_format,
                                    embeddings_model)
    
    saveEncodedTweets(encoded_tweets,
                      encoding_format, 
                      file_name=f'{n_folds}FOLDS/{embeddings_label}_F{K}_P{prep_format}.data')

## saveEncodedTweets()

In [ ]:
# save encoded tweets into disk
import pickle
def saveEncodedTweets(encoded_tweets, encoding_format, file_name):
  path = './dataset_files/Encoded/{}/{}'.format(encoding_format, file_name)
  print("Saving data to:")
  print(path)

  with open(path, 'wb') as filehandle:
      # store the encoded documents as binary data
      pickle.dump(encoded_tweets, filehandle)
      
  print("\nCOMPLETE\n")

# FT1

In [ ]:
%time ft = ft_model(model_id=1)

loading fatText model...


CPU times: user 3.76 s, sys: 5.63 s, total: 9.38 s
Wall time: 46.5 s


## encoding tweets

## Single-Vector encodings

### Train-set

In [ ]:
encodeDataFolds(train_df, 
                n_folds=7, 
                prep_format=2, 
                encoding_format='SINGLE-VEC',
                embeddings_model=ft,
                embeddings_label='FT1')

### Test-set

In [ ]:
prep_format = 2

test_tweets = test_df['preprocessed_text_{}'.format(prep_format)]
encoded_test_tweets = transformTweets(test_tweets, 
                                  format='SINGLE-VEC',
                                  ft_model=ft)
  
saveEncodedTweets(encoded_test_tweets,
                  encoding_format='SINGLE-VEC', 
                  file_name='FT1_TEST_P{}.data'.format(prep_format))

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/FT1_TEST_P2.data

COMPLETE



In [ ]:
test_tweets[0]

In [ ]:
type(encoded_test_tweets)

numpy.ndarray

In [ ]:
encoded_test_tweets.shape

(1600, 300)

## Embeddings-Sequence encodings

## Train-set

In [ ]:
encodeDataFolds(train_df, 
                n_folds=7, 
                prep_format=2, 
                encoding_format='EMB-SEQ',
                embeddings_model=ft,
                embeddings_label='FT1')

## Test-set

In [ ]:
prep_format = 2

test_tweets = test_df['preprocessed_text_{}'.format(prep_format)]
encoded_test_tweets = transformTweets(test_tweets, 
                                  format='EMB-SEQ',
                                  ft_model=ft)
  
saveEncodedTweets(encoded_test_tweets,
                  encoding_format='EMB-SEQ', 
                  file_name='FT1_TEST_P{}.data'.format(prep_format))

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/FT1_TEST_P2.data

COMPLETE



In [ ]:
type(encoded_test_tweets)

numpy.ndarray

In [ ]:
encoded_test_tweets.shape

(1600, 55, 300)

# FT2

In [ ]:
%time ft = ft_model(model_id=2)

loading fatText model...
CPU times: user 3.94 s, sys: 13.8 s, total: 17.8 s
Wall time: 1min 5s


## encoding tweets

## Single-Vector encodings

### Train-set

In [ ]:
encodeDataFolds(train_df, 
                n_folds=7, 
                prep_format=2, 
                encoding_format='SINGLE-VEC',
                embeddings_model=ft,
                embeddings_label='FT2')

### Test-set

In [ ]:
prep_format = 2

test_tweets = test_df['preprocessed_text_{}'.format(prep_format)]
encoded_test_tweets = transformTweets(test_tweets, 
                                  format='SINGLE-VEC',
                                  ft_model=ft)
  
saveEncodedTweets(encoded_test_tweets,
                  encoding_format='SINGLE-VEC', 
                  file_name='FT2_TEST_P{}.data'.format(prep_format))

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/FT2_TEST_P2.data

COMPLETE



In [ ]:
type(encoded_test_tweets)

numpy.ndarray

In [ ]:
encoded_test_tweets.shape

(1600, 300)

## Embeddings-Sequence encodings

## Train-set

In [ ]:
encodeDataFolds(train_df, 
                n_folds=7, 
                prep_format=2, 
                encoding_format='EMB-SEQ',
                embeddings_model=ft,
                embeddings_label='FT2')

## Test-set

In [ ]:
prep_format = 2

test_tweets = test_df['preprocessed_text_{}'.format(prep_format)]
encoded_test_tweets = transformTweets(test_tweets, 
                                  format='EMB-SEQ',
                                  ft_model=ft)
  
saveEncodedTweets(encoded_test_tweets,
                  encoding_format='EMB-SEQ', 
                  file_name='FT2_TEST_P{}.data'.format(prep_format))

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/FT2_TEST_P2.data

COMPLETE



In [ ]:
type(encoded_test_tweets)

numpy.ndarray

In [ ]:
encoded_test_tweets.shape

(1600, 55, 300)

# FT3

In [ ]:
%time ft = ft_model(model_id=3)

loading fatText model...
CPU times: user 3.2 s, sys: 7.91 s, total: 11.1 s
Wall time: 2min 25s


## Encoding tweets

## Single-Vector encodings

### Train-set

In [ ]:
encodeDataFolds(train_df, 
                n_folds=7, 
                prep_format=2, 
                encoding_format='SINGLE-VEC',
                embeddings_model=ft,
                embeddings_label='FT3')

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F0_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F1_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F2_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F3_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F4_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F5_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/7FOLDS/FT3_F6_P2.data

CO

### Test-set

In [ ]:
prep_format = 2

test_tweets = test_df['preprocessed_text_{}'.format(prep_format)]
encoded_test_tweets = transformTweets(test_tweets, 
                                  format='SINGLE-VEC',
                                  ft_model=ft)
  
saveEncodedTweets(encoded_test_tweets,
                  encoding_format='SINGLE-VEC', 
                  file_name='FT3_TEST_P{}.data'.format(prep_format))

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/SINGLE-VEC/FT3_TEST_P2.data

COMPLETE



In [ ]:
type(encoded_test_tweets)

numpy.ndarray

In [ ]:
encoded_test_tweets.shape

(1600, 300)

## Embeddings-Sequence encodings

## Train-set

In [ ]:
encodeDataFolds(train_df, 
                n_folds=7, 
                prep_format=2, 
                encoding_format='EMB-SEQ',
                embeddings_model=ft,
                embeddings_label='FT3')

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F0_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F1_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F2_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F3_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F4_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F5_P2.data

COMPLETE

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/7FOLDS/FT3_F6_P2.data

COMPLETE



## Test-set

In [ ]:
prep_format = 2

test_tweets = test_df['preprocessed_text_{}'.format(prep_format)]
encoded_test_tweets = transformTweets(test_tweets, 
                                  format='EMB-SEQ',
                                  ft_model=ft)
  
saveEncodedTweets(encoded_test_tweets,
                  encoding_format='EMB-SEQ', 
                  file_name='FT3_TEST_P{}.data'.format(prep_format))

Transforming tweets...
Tweets were succesfully transformed.
Saving data to:
./dataset_files/Encoded/EMB-SEQ/FT3_TEST_P2.data

COMPLETE



In [ ]:
type(encoded_test_tweets)

numpy.ndarray

In [ ]:
encoded_test_tweets.shape

(1600, 55, 300)